# Submission

In [1]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
import os
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import utils
%matplotlib inline

import math

In [3]:
def getImgPath(img_dir, paths):
    img_paths = {}
    for path in paths:
        files = os.listdir(os.path.join(img_dir, path))
        files = sorted(files)
        for f in files:
            if f.split(".")[1] == "mhd":
                filename = f.split(".")[0]
                absolute_path = os.path.join(img_dir, os.path.join(path, f))
                img_paths[filename] = absolute_path
    return img_paths
    
bboxes_dir = "/home/htang6/workspace/TCAI17/DSB2017/TCAI/training/detector-seg/results/0530/res18/submit/bbox"
filenames = np.load("/home/htang6/data/1x1x1/test/filenames.npy")
img_dir = "/home/htang6/data"
spacings = np.load(os.path.join("/home/htang6/data/1x1x1/test/", "spacing"))
spacings = spacings.as_matrix()
test_directory = ["test_subset00/", "test_subset01/", "test_subset02/","test_subset03/", "test_subset04/"]
save_dir = "/home/htang6/workspace/TCAI17/DSB2017/TCAI/training/detector-seg/results/0530/res18/submit/submission"

img_paths = getImgPath(img_dir, test_directory)

IOError: [Errno 2] No such file or directory: '/home/htang6/data/1x1x1/test/filenames.npy'

In [3]:
def generateSubmission(filenames, k, bboxes_dir, img_dir, spacings, save_dir):
    """
    Generate the submission.csv for TCAI17
    
    filenames: All the patients name for testing, eg. "LKDS-00002"
    k: Top k predicted bboxes per patient
    bboxes_dir: The directory for stroing all the bounded boxes, all the bounded boxes should be named "LKDS-00002_pbb.npy"
    img_dir: The directory of the original .mhd, because we need the original origin information
    spacings: Numpy array of all the spacing information of patients
    save_dir: Where do you want to save the submission.csv file
    """
    submission = []

    for patient in filenames:
        pbb = np.load(os.path.join(bboxes_dir, "%s_pbb.npy" % (patient)))
        pbb = pbb[pbb[:, 0].argsort()][::-1][:k]
        spacing = spacings[spacings[:, 0] == patient][:, [1, 2, 3]]
        #print spacing

        _, numpyOrigin, _ = utils.load_itk_image(os.path.join(img_dir, "%s.mhd" % (patient)))

        for p in pbb:
            voxelCoord = p[[1, 2, 3]]
            worldCoord = utils.voxelToWorldCoord(voxelCoord, numpyOrigin, spacing)
            submission.append([patient, worldCoord[0][2], worldCoord[0][1], worldCoord[0][0], p[0]])
        
        submission = pd.DataFrame(submission, columns = ["seriesuid", "coordX", "coordY", "coordZ", "probability"])
        print "Saving submission..."
        submission.to_csv(save_dir, sep=',', index=False)


    
k = 10

submission = []

for patient in filenames:
    pbb = np.load(os.path.join(bboxes_dir, "%s.npy_pbb.npy" % (patient)))
    pbb = pbb[pbb[:, 0].argsort()][::-1][:k]
    spacing = spacings[spacings[:, 0] == patient][:, [1, 2, 3]]
    #print spacing
    
    img_path = img_paths[patient]
    _, numpyOrigin, _ = utils.load_itk_image(img_path)
    
    
    #pbb = np.insert(pbb, patient, 0, axis = 0)
    for p in pbb:
        voxelCoord = p[[1, 2, 3]]
#         print voxelCoord
#         print p
        worldCoord = utils.voxelToWorldCoord(voxelCoord, numpyOrigin, spacing)
#         print worldCoord
#         print utils.worldToVoxelCoord(worldCoord, numpyOrigin, spacing)
        submission.append([patient, worldCoord[0][2], worldCoord[0][1], worldCoord[0][0], p[0]])

submission = pd.DataFrame(submission, columns = ["seriesuid", "coordX", "coordY", "coordZ", "probability"])
print "Saving submission..."
submission.to_csv(save_dir, sep=',', index=False)
#print submission
#submission.to_csv(save_dir, sep=',')

Saving submission...


# Check coords
double check whether the world to voxel and voxel to world is the same

In [14]:
def getImgPath(img_dir, paths):
    img_paths = {}
    for path in paths:
        files = os.listdir(os.path.join(img_dir, path))
        files = sorted(files)
        for f in files:
            if f.split(".")[1] == "mhd":
                filename = f.split(".")[0]
                absolute_path = os.path.join(img_dir, os.path.join(path, f))
                img_paths[filename] = absolute_path
    return img_paths
    
bboxes_dir = "/home/htang6/workspace/TCAI17/DSB2017/TCAI/training/detector/results/res18_64/bboxes/bbox"
filenames = np.load("/home/htang6/data/1x1x1/val/filenames.npy")
img_dir = "/home/htang6/data"
spacings = np.load(os.path.join("/home/htang6/data/1x1x1/val/", "spacing"))
spacings = spacings.as_matrix()
test_directory = ["val_subset00/", "val_subset01/", "val_subset02/","val_subset03/", "val_subset04/"]

cands_val = pd.read_csv("/home/htang6/data/csv/val/annotations.csv")
labels = cands_val.as_matrix()

img_paths = getImgPath(img_dir, test_directory)

# for patient in filenames:
#     print "Current patient: ", patient, "img_path: ", img_paths[patient]
#     lbb = np.load(os.path.join(bboxes_dir, "%s.npy_lbb.npy" % (patient)))
#     spacing = spacings[spacings[:, 0] == patient][:, [1, 2, 3]]
#     #print spacing
    
#     patient_labels = labels[labels[:, 0] == patient]
#     patient_labels = patient_labels[:, [3, 2, 1, 4]]
    
#     img_path = img_paths[patient]
#     _, numpyOrigin, _ = utils.load_itk_image(img_path)
    
#     for l in patient_labels:
#         print l
#         worldCoord_ori = l[:-1]
#         voxelCoord_ori = utils.worldToVoxelCoord(worldCoord_ori, numpyOrigin, spacing)
        
#         print "1: ", worldCoord_ori
#         print "2: ", voxelCoord_ori
#         break
#     break

for i, patient in enumerate(filenames):
    if i <= 40:
        continue
    if i >= 50:
        break
    print "Current patient: ", patient, "img_path: ", img_paths[patient]
    lbb = np.load(os.path.join(bboxes_dir, "%s.npy_lbb.npy" % (patient)))
    spacing = spacings[spacings[:, 0] == patient][:, [1, 2, 3]]
    #print spacing
    
    patient_labels = labels[labels[:, 0] == patient]
    patient_labels = patient_labels[:, [3, 2, 1, 4]]
    
    img_path = img_paths[patient]
    _, numpyOrigin, _ = utils.load_itk_image(img_path)
    
    for l in lbb:
        voxelCoord = l[[0, 1, 2]]
        worldCoord = utils.voxelToWorldCoord(voxelCoord, numpyOrigin, spacing)     
        
        print voxelCoord
        print "Transformed: ", worldCoord
        print "Ground truth: ", patient_labels
        print

Current patient:  LKDS-00177 img_path:  /home/htang6/data/val_subset01/LKDS-00177.mhd
[  78.99991608  205.74456787  304.71276855]
Transformed:  [[886.0999160766602 162.2446556868409 87.21289861197027]]
Ground truth:  [[886.0999145510001 162.244650136 87.212890625 7.30064369645]]

Current patient:  LKDS-00190 img_path:  /home/htang6/data/val_subset01/LKDS-00190.mhd
[ 219.99998474  169.07345581   68.71386719]
Transformed:  [[597.7999847412109 104.07351220084627 -115.28610989467694]]
Ground truth:  [[597.799987793 104.073510363 -115.28610751299999 10.2124990659]]

Current patient:  LKDS-00197 img_path:  /home/htang6/data/val_subset01/LKDS-00197.mhd
[ 173.          240.92169189   94.99662018]
Transformed:  [[253.1 -78.73622407865099 -88.66134668958559]]
Ground truth:  [[253.09999847400002 -78.7362232178 -88.6613503406 18.0361746161]]

Current patient:  LKDS-00201 img_path:  /home/htang6/data/val_subset01/LKDS-00201.mhd
[  62.43458939  275.2467041   247.97520447]
Transformed:  [[-691.099999

# Submission 2-seg

In [2]:
def voxelToWorldCoord(voxelCoord, origin, spacing):
    worldCoord = voxelCoord * spacing
    worldCoord += origin
    return worldCoord

def worldToVoxelCoord(worldCoord, origin, spacing):
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

def s_to_p(s):
    """Convert scores array to probabilities."""
    from torch.autograd import Variable
    from torch import from_numpy
    from torch.nn import Sigmoid
    m = Sigmoid()
    input = Variable(from_numpy(s))
    return  m(input).data.numpy()

def generate_sub_segmentation(filenames, k, bboxes_dir, prep_dir, pred_dir=None, save_dir=None):
    """
    Generate the submission.csv for TCAI17
    
    filenames: All the patients name for testing, eg. "LKDS-00002"
    k: Top k predicted bboxes per patient
    bboxes_dir: The directory for stroing all the bounded boxes, 
    all the bounded boxes should be named "LKDS-00002_pbb.npy"
    prep_dir: The directory of the preprocessed images
    save_dir: Where do you want to save the submission.csv file
    """
    submission = []

    for patient in filenames:
        pbb = np.load(os.path.join(bboxes_dir, "%s_pbb.npy" % (patient)))
        
        if len(pbb) < 1:
            continue
        
#         pred = np.load(os.path.join(pred_dir, "%s.npy_pred.npy" % (patient)))
#         pbb = np.concatenate([pbb,pred], axis=1)
        pbb[:,0] = s_to_p(pbb[:, 0])
#         pbb[:,-1] = s_to_p(pbb[:, -1])
        pbb = pbb[pbb[:,0] > 0.5]
        
#         mask = pred >= 0.5
#         pbb = pbb[mask[:, 0], ...]
        
#         pbb = pbb[pbb[:, 0].argsort()][::-1][:k]
        spacing = np.load(os.path.join(prep_dir, patient + '_spacing.npy'))
        ebox_origin = np.load(os.path.join(prep_dir, patient + '_ebox_origin.npy'))
        origin = np.load(os.path.join(prep_dir, patient + '_origin.npy'))
        #print spacing

        for p in pbb:
            ebox_coord = p[[1, 2, 3]]
            whole_img_coord = ebox_coord + ebox_origin
            worldCoord = voxelToWorldCoord(whole_img_coord, origin, spacing)
            submission.append([patient, worldCoord[2], worldCoord[1], worldCoord[0], p[0]])

    submission = pd.DataFrame(submission, columns = ["seriesuid", "coordX", "coordY", "coordZ",
                                                     "probability"])
#     import torch
#     m = torch.nn.Sigmoid()
#     scores = np.array(submission['probability'])
#     input = torch.autograd.Variable(torch.from_numpy(scores))
#     probs = m(input).data.numpy()
#     submission['probability'] = probs
    
    if save_dir != None:
        print "Saving submission..."
        submission.to_csv(save_dir, sep=',', index=False)
    return submission

def submit_wrapper(results_dir, sub, k=10):
    """Return dataframe of predictions and save csv file.
    
    sub: True if using submission (test) set; save csv as 'sub.csv'. 
         False if using val set; save csv as 'val.csv'.
    """
    prep_dir = '/home/htang6/data/1x1x1_seg'
    BASE = '/home/danielrk/tc/nod/training/detector/'
    bboxes_dir= os.path.join(BASE, 'results', results_dir, 'bbox/')
    if sub: 
        bboxes_dir = os.path.join(bboxes_dir, 'sub')
        filenames = np.load(os.path.join(BASE,'filenames_test.npy'))
        save_dir = os.path.join(BASE, 'results', results_dir, 'sub.csv')
    else:
        bboxes_dir = os.path.join(bboxes_dir, 'val')
        filenames = np.load(os.path.join(BASE,'filenames_val.npy'))
        save_dir = os.path.join(BASE, 'results', results_dir, 'val.csv')
    return generate_sub_segmentation(filenames, k, bboxes_dir, prep_dir, save_dir)

In [15]:
prep_dir = '/home/danielrk/lung/input/tianchi_prep_HU_pres'
bboxes_dir = "/home/htang6/workspace/detector/results/test/0/bbox/test_130"
pred_dir = "/home/htang6/workspace/TCAI17/DSB2017/TCAI/training/classifier-seg/results/0622/VGG13_shortcut/results/test2"
save_dir = "/home/htang6/submission.csv"
filenames = np.load("/home/htang6/workspace/detector/filenames_test2.npy")
blacklist = ["LKDS-00322", "LKDS-00385"]

print filenames
print generate_sub_segmentation(filenames, 30, bboxes_dir, prep_dir, save_dir=save_dir)

['LKDS-01004' 'LKDS-01005' 'LKDS-01009' 'LKDS-01011' 'LKDS-01013'
 'LKDS-01014' 'LKDS-01021' 'LKDS-01023' 'LKDS-01025' 'LKDS-01026'
 'LKDS-01029' 'LKDS-01030' 'LKDS-01032' 'LKDS-01033' 'LKDS-01034'
 'LKDS-01037' 'LKDS-01038' 'LKDS-01039' 'LKDS-01041' 'LKDS-01042'
 'LKDS-01043' 'LKDS-01044' 'LKDS-01047' 'LKDS-01048' 'LKDS-01049'
 'LKDS-01050' 'LKDS-01053' 'LKDS-01054' 'LKDS-01056' 'LKDS-01058'
 'LKDS-01061' 'LKDS-01062' 'LKDS-01063' 'LKDS-01064' 'LKDS-01065'
 'LKDS-01066' 'LKDS-01067' 'LKDS-01068' 'LKDS-01069' 'LKDS-01071'
 'LKDS-01073' 'LKDS-01075' 'LKDS-01077' 'LKDS-01078' 'LKDS-01081'
 'LKDS-01083' 'LKDS-01084' 'LKDS-01085' 'LKDS-01086' 'LKDS-01087'
 'LKDS-01088' 'LKDS-01089' 'LKDS-01091' 'LKDS-01092' 'LKDS-01093'
 'LKDS-01094' 'LKDS-01095' 'LKDS-01098' 'LKDS-01099' 'LKDS-01100'
 'LKDS-01102' 'LKDS-01103' 'LKDS-01104' 'LKDS-01107' 'LKDS-01108'
 'LKDS-01109' 'LKDS-01110' 'LKDS-01112' 'LKDS-01113' 'LKDS-01114'
 'LKDS-01115' 'LKDS-01118' 'LKDS-01120' 'LKDS-01122' 'LKDS-01123'
 'LKDS-011